<a href="https://colab.research.google.com/github/ngyamfi82/emeritus/blob/master/malware_behaviourV2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In v2, I noticed that the ember features was built with an old versionof lief(0.9.0). Due to this our preprocessing function was failing. fix this issue we have to recompute the ember files from the original datset [we have extracted] and push it into our preprocessing pipeline


In [ ]:
pip install git+https://github.com/elastic/ember.git

  Cloning https://github.com/elastic/ember.git to /tmp/pip-req-build-jmjmwmyw
  Running command git clone --filter=blob:none --quiet https://github.com/elastic/ember.git /tmp/pip-req-build-jmjmwmyw
  Resolved https://github.com/elastic/ember.git to commit d97a0b523de02f3fe5ea6089d080abacab6ee931
  Preparing metadata (setup.py) ... done
  Created wheel for ember: filename=ember-0.1.0-py3-none-any.whl size=13050 sha256=2c3e3ec33b8d18ee70e1aaf888460f03c0486a312152a084e69bbeff8ebcd68f
  Stored in directory: /tmp/pip-ephem-wheel-cache-k62gi34t/wheels/7a/af/81/7e3bd4d43fd62c37273aa84e0720752df8dbc9c43700279961
Successfully built ember


In [ ]:
pip install lief

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.9 MB/s eta 0:00:00


In [ ]:
pip install ember lightgbm

In [ ]:
# !python -m pip install multiprocessing

In [ ]:
pip install git+https://github.com/NeuromorphicComputationResearchProgram/ClarAVy


  Cloning https://github.com/NeuromorphicComputationResearchProgram/ClarAVy to /tmp/pip-req-build-e8w0meuj
  Running command git clone --filter=blob:none --quiet https://github.com/NeuromorphicComputationResearchProgram/ClarAVy /tmp/pip-req-build-e8w0meuj
  Resolved https://github.com/NeuromorphicComputationResearchProgram/ClarAVy to commit f496ce84e4bc037215ba0152b236aefdf52f8cca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for claravy: filename=claravy-1.0.0-py3-none-any.whl size=101798 sha256=32d4c607934d486ae7110c3010580cc65dd16f02f82e1cb532eb0eae39188ec2
  Stored in directory: /tmp/pip-ephem-wheel-cache-nf8cbde3/wheels/ba/d9/e6/c629c37334dcf28fe2a45c5f69fd7179c92f93cf4eddc6cdd8
Successfully built claravy


LOADING REQUIRED LIBRARIES

In [ ]:
#%% Import libraries
import ember
import lief
# import claravy

import lightgbm as lgb
import numpy as np
import multiprocessing
import json
import os
from IPython import get_ipython

import pickle



import warnings
warnings.filterwarnings("ignore")

Genrating new ember files from our extracted dataset.

In [ ]:
#%% Load data
ember_meta_dir = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/ember_metadata"
maldict_train_file = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/train/maldict_category_train.jsonl"
maldict_test_file = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/test/maldict_category_test.jsonl"



train_path = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/train"

test_path = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/test"

In [ ]:
train_path = os.path.join(ember_meta_dir, "ember_metadata_train/train_features.jsonl")
test_path = os.path.join(ember_meta_dir, "ember_metadata_test/test_features.jsonl")

In [ ]:

# def extract_features(file_path):
#     try:
#         binary = lief.parse(file_path)
#         if not binary:
#             return None
#         # Extract the features using ember
#         extractor = ember.PEFeatureExtractor()
#         features = extractor.process(binary)
#         return features
#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return None

# # Directory containing your original PE files
# data_directory = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/train/"
# pe_files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) ]#if f.endswith('.exe')]

# # Open a JSON Lines file for writing
# with open("/content/drive/MyDrive/Colab Notebooks/MalwareDetection/ember_metadata/Train_ember_features.jsonl", "w") as jsonl_file:
#     for pe_file in pe_files:
#         features = extract_features(pe_file)
#         if features is not None:
#             # Convert features to a dictionary if needed
#             if isinstance(features, np.ndarray):
#                 features = features.tolist()
#             # Write the features to the JSON Lines file
#             jsonl_file.write(json.dumps(features) + "\n")

# print("Features have been written to ember_features.jsonl")

In [ ]:
# # Define a function to extract features from a PE file [test]
# def extract_features(file_path):
#     try:
#         binary = lief.parse(file_path)
#         if not binary:
#             return None
#         # Extract the features using ember
#         extractor = ember.PEFeatureExtractor()
#         features = extractor.process(binary)
#         return features
#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return None

# # Directory containing your original PE files
# data_directory = "/content/drive/MyDrive/Colab Notebooks/MalwareDetection/test/1/"
# pe_files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) ]#if f.endswith('.exe')]

# # Open a JSON Lines file for writing
# with open("/content/drive/MyDrive/Colab Notebooks/MalwareDetection/ember_metadata/Test_ember_features.jsonl", "w") as jsonl_file:
#     for pe_file in pe_files:
#         features = extract_features(pe_file)
#         if features is not None:
#             # Convert features to a dictionary if needed
#             if isinstance(features, np.ndarray):
#                 features = features.tolist()
#             # Write the features to the JSON Lines file
#             jsonl_file.write(json.dumps(features) + "\n")

# print("Features have been written to ember_features.jsonl")

In [ ]:
def get_vec(jsonl):
    """Vectorize a JSON line with EMBER raw features.
    Returns: (md5, vector)
    """
    ember_meta = json.loads(jsonl)
    md5 = ember_meta["md5"]
    extractor = ember.PEFeatureExtractor()
    vec = extractor.process_raw_features({"entry": [[ember_meta['entry']]]})  # Update this line
    return md5, vec


In [ ]:
# def get_vec(jsonl):
#     """Vectorize a JSON line with EMBER raw features.

#     Returns:
#     (md5, vector)
#     """
#     ember_meta = json.loads(jsonl)
#     md5 = ember_meta["md5"]


#     extractor = ember.PEFeatureExtractor()
#     vec = extractor.process_raw_features(ember_meta)
#     return md5, vec

In [ ]:
def get_tags(tag_path):
    """Make a dictionary mapping MD5s to ClarAVy tags."""
    md5_tags = {}
    with open(tag_path, "r") as f:
        for jsonl in f:
            entry = json.loads(jsonl.strip())
            md5 = entry["md5"]
            tags = [rank[0] for rank in entry["ranking"]]
            md5_tags[md5] = tags
    return md5_tags

### DATA PREPROCESSING AND FEATURE ENGINEERING

In [ ]:
with open(train_path, "r") as f:
    train_meta = [line.strip() for line in f]
with open(test_path, "r") as f:
    test_meta = [line.strip() for line in f]

In [ ]:
#%% Read train/test tags
train_md5_tags = get_tags(maldict_train_file)
test_md5_tags = get_tags(maldict_test_file)

In [ ]:
# import json

# def get_vec(ember_meta):
#   raw_obj = json.loads(ember_meta)
#   entry_name_hashed = FeatureHasher(50, input_type="string").transform([[raw_obj['entry']]]).toarray()[0]
#   return entry_name_hashed

# pool = multiprocessing.Pool(4)  # adjust num_processes as needed
# train_md5_vecs = list(pool.imap(get_vec, train_meta))
# test_md5_vecs = list(pool.imap(get_vec, test_meta))

In [ ]:
#%% Vectorize EMBER metadata
pool = multiprocessing.Pool(10)  # adjust num_processes as needed
train_md5_vecs = list(pool.imap(get_vec, train_meta))
test_md5_vecs = list(pool.imap(get_vec, test_meta))

NameError: name 'train_meta' is not defined

In [ ]:
#%% Get sizes of train and test set
num_train = len(train_md5_vecs)
num_test = len(test_md5_vecs)
vec_dim = len(train_md5_vecs[0][1])
num_labels = len(sorted_tags)

MODEL TRAINING AND EVALUATION


### MODEL SELECTION

In [ ]:
#%% Get X and y for train set
X_train = np.zeros((num_train, vec_dim), dtype=np.float)
y_train = np.zeros((num_train, num_labels))
for i, (md5, vec) in enumerate(train_md5_vecs):
    labels = [tag_labels[tag] for tag in train_md5_tags[md5]]
    X_train[i,] = vec
    for j in labels:
        y_train[i,j] = 1

In [ ]:
#%% Get X and y for test set
X_test = np.zeros((num_test, vec_dim), dtype=np.float)
y_test = np.zeros((num_test, num_labels), dtype=np.float)
for i, (md5, vec) in enumerate(test_md5_vecs):
    labels = [tag_labels[tag] for tag in test_md5_tags[md5]]
    X_test[i,] = vec
    for j in labels:
        y_test[i,j] = 1

### MODEL TRAINING AND EVALUATION

In [ ]:
#%% Train OvR classifier on each tag
y_pred = np.zeros((num_test, num_labels))
for j, tag in enumerate(sorted_tags):
    print("Training classifiers on tag: {}".format(tag))

      # Get train and test sets for fold
      y_train_tag = y_train[:, j]
      y_test_tag = y_test[:, j]

      # Train LightGBM classifier
      train_dataset = lgb.Dataset(X_train, y_train_tag)
      test_dataset = lgb.Dataset(X_test, y_test_tag)
      clf = lgb.train(params, train_dataset)

      # Get predictions and compute accuracy
      predictions = clf.predict(X_test)
      y_pred[:,j] = predictions


In [ ]:
#%% Print results
print("Precision\t{} (micro)\t{} (weighted)".format(p_micro, p_avg))
print("Recall\t{} (micro)\t{} (weighted)".format(r_micro, r_avg))
print("F1-Score\t{} (micro)\t{} (weighted)".format(f1_micro, f1_avg))
print("ROC AUC\t{} (micro)\t{} (weighted)".format(micro_auc, weighted_auc))

In [ ]:
pickle.export("malwareBehaviorDetection.pkl")

In [ ]:
model  = "malwareBehaviorDetection.pkl"

In [ ]:
model.predict("contract.exe")

In [ ]:
["contract.exe", [['worm'],[13]]]

In [ ]:
["contract.exe",[["worm",[13],"spyware"[12]]]]